<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 10.3 - Using AWS for cluster computing

### Introduction

The purpose of this lab is to gain exposure to cluster computing, often necessary when datasets become too large to manage on a local machine. You will learn how to work with a large dataset through AWS - in particular using an EMR cluster and the PySpark Python library.

Note that this notebook will need to be loaded into an AWS SageMaker instance in order for the code to run succesfully. It will not work on your laptop.

**Important**: To prevent excessive billing costs, you will need to shut down your instances when you finish working on AWS for the day - see instructions at the bottom of this page. Please give yourself 10 minutes at the end of the day for this. 

After you have logged into the AWS console you will be able to see the billing cost explorer at https://console.aws.amazon.com/billing/home?/costexplorer#/costexplorer


### 1. Connecting to AWS via AWS Educate

Sign into the AWS console as done in Lab 10.1:

a) Sign in to https://www.awseducate.com/signin/SiteLogin and navigate to the "Classrooms & Credits" tab, then "Request or go to an AWS Educate Classroom".

b) Select "My classrooms" and then the "Go to classroom" link that appears under "Courses where I am a Student".

c) You will be redirected to a https://labs.vocareum.com page. Click "AWS Console" and you will be directed to your AWS console.

### 2. Setting up a SageMaker instance connected to an EMR cluster

Follow the instructions in the link below to create a SageMaker notebook instance linked to a Spark  Elastic Map Reduce (EMR) cluster. This involves configuring some security groups to allow SageMaker to communicate with the cluster. Finally the notebook is connected to Spark by using Livy, a REST API for interacting with Spark.

https://aws.amazon.com/blogs/machine-learning/build-amazon-sagemaker-notebooks-backed-by-spark-in-amazon-emr/

**Optional**: To use up fewer credits, use a cheaper instance in EMR than the `m5.xlarge` default given. One possibility is using `m4.large` instead (but it will be slower to run). A full list of instances and pricing is given at https://aws.amazon.com/emr/pricing/.

While working through this setup it will be convenient to have a text editor (e.g Notepad) open to make note of the following details about the cluster that will be needed when setting up SageMaker. Have separate browser tabs open for the EMR cluster console and for SageMaker.

- Private IP Address: e.g. `172.31.46.73` 
- VPC name: `vpc-..... (<ip address>)`
- EC2 Subnet: `subnet-..... default in us-...`
- EMR master's security group: `sg-... (ElasticMapReduce-master)`

### 3. Setting up a Spark session

Upload this notebook into your SageMaker instance setting the kernel to `sparkmagic` where requested and then run the cell below. You will be able to proceed if there is no error.

In [4]:
%%info

UsageError: Cell magic `%%info` not found.


In [5]:
from pyspark.sql import SparkSession

ModuleNotFoundError: No module named 'pyspark'

The SparkSession class allows one to create a Spark DataFrame.

In [ ]:
spark = SparkSession.builder.appName("Temperature Analysis").getOrCreate()

### 4. Working with a large dataset

We will work with a public dataset from the Registry of Open Data on AWS. GSOD (Global Surface Summary of the Day) contains daily weather measurements from over 9000 weather stations dating back to 1901 (with more complete data since the 1970s). It is located in s3 at s3://aws-gsod. More details about the data can be found at https://aws.amazon.com/public-datasets/gsod/

Note that some of the cells to be run below start with %%local. This means those cells are run locally (on the SageMaker instance) rather than on the cluster.

In [1]:
%%help

UsageError: Cell magic `%%help` not found.


Let us read in the data from all weather stations for the year 2000. Here programming will primarily use the PySpark library.

In [2]:
weatherdata2000 = spark.read.option("header", "true").csv('s3://aws-gsod/2000/*.csv')

NameError: name 'spark' is not defined

In [3]:
type(weatherdata2000)

NameError: name 'weatherdata2000' is not defined

The printSchema() method is useful to list the types of each column.

In [ ]:
weatherdata2000.printSchema()

Use the count() method to show that this dataset has over 2.5 million rows.

In [ ]:
weatherdata2000.count()

Show a few rows

In [ ]:
weatherdata2000.show(5)

Here is a neater way to display the same data:

In [ ]:
weatherdata2000.show(n=5, truncate=False, vertical=True)

The following cell creates a Pandas dataframe that contains data for one weather station.

In [ ]:
%%spark -o syd_pdf
syd_pdf = weatherdata2000.filter(weatherdata2000.USAF == '947680')

Here is a reference list of a few weather station locations by USAF, in case you would like to try another location.

947680 - Sydney

948680 - Melbourne

486980 - Singapore

725033 - New York

583620 - Shanghai

037683 - London

In [ ]:
%%local
type(syd_pdf)

We see it is now a familiar Pandas dataframe. Being of manageable size, one may work with it using Pandas.

In [ ]:
%%local
syd_pdf.shape

**Exercise**: Create a line chart of this dataframe using Date on the x axis and Mean_Temp on the y axis. Note that the cell below is enough to get us started.

**Bonus**: Convert the temperature to degrees Celcius before plotting.

In [ ]:
%%local
syd_pdf

PySpark has `groupby`, `filter`, `select` and `describe` as commonly used methods. For example the following cell summarises the Max_Temp column in weatherdata2000.

In [ ]:
weatherdata2000.describe(['Max_Temp']).show()

Note the max does not seem to exceed 100 because the values of this field are still strings. We shall use the following code to convert the types of a few columns.

In [ ]:
from pyspark.sql.functions import avg
from pyspark.sql.functions import col

In [ ]:
cols = ['Max_Temp', 'Mean_Temp', 'Min_Temp']

for col_name in cols:
    weatherdata2000 = weatherdata2000.withColumn(col_name, col(col_name).cast('float'))

Verify that this did as intended:

In [ ]:
weatherdata2000.describe(['Max_Temp']).show()

**Exercise**: How many records in weatherdata2000 have Country_Code of 'AS' (Australia)?

**Exercise**: Write a groupby query to determine the 10 hottest weather stations by average maximum temperature (i.e. averaged over the year for each station ID). Show four column headings: ID, Latitude, Longitude and the average maximum temperature.

**Optional**: If time permits look up PySpark documentation to investigate other ways to analyse the data.

### 5. Closing AWS instances

**Firstly, ensure you have saved your notebook locally** before performing this. Also give yourself sufficient time at the end of the day (10 minutes) to complete this.

Stop your SageMaker instance by returning to the "Notebook instances" page, selecting your instance and then under Actions select "Stop". Then once the status shows "Stop" you may delete this instance. 

Terminate the EMR cluster by selecting it in the console and then pressing the Terminate button. A window might show that your cluster has termination protection turned on. Select "Turn off protection", select the tick button and then hit the red terminate button.

Once the EMR, EC2 and SageMaker consoles are clear of active instances, you may sign out of AWS.


After a couple of days log into https://console.aws.amazon.com/billing/home?/costexplorer#/costexplorer via AWS Educate to see if there are still running charges.

### Further reference

https://github.com/aws/amazon-sagemaker-examples



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



